
# Results Profiling, Colour Picker Notebook

Developed by 2Pi Software

## Function of the SSWP Colour Picker

The SSWP Colour Picker examines sets of images to identify pixels with colours falling in a specified range. To assist interaction of determining the colour range, a sample colour 'Swatch' is referenced.

First, some helper libraries are loaded.

In [ ]:
# --- Initialisation ----

# For dev, allow refresh of support lib
import importlib

import ASDC_API_support
import ASDC_API_helper
importlib.reload(ASDC_API_support)
importlib.reload(ASDC_API_helper)
print("Lib loaded")

# Useful links to remember
site_url = "https://dev2pi.sswp.cosinecrm.com.au"
base_url = site_url + "/sswpapps-api/"
auth_url = base_url + "auth"



# Establish API authorisation

## Setup your ASDC user access

1. go to https://dev2pi.sswp.cosinecrm.com.au
2. log in to your SSWP account 
3. go to SSWP Apps > Manage account
4. set an App password using the Update Password interface in the middle of the screen



## Obtain a token for API access
1. go to https://dev2pi.sswp.cosinecrm.com.au/sswpapps-api/auth
2. log in with your ASDC user name and password
3. copy out the token, and paste it into the following cell as a string
4. (revisit the link to refresh or revoke your token at any time)


# Use a token to access API endpoints



In [ ]:

api_helper = ASDC_API_helper.Helper()
asdc_api = api_helper.asdc_api



## Workflow: Choose upload, then profile results.



In [ ]:

api_helper.select_run()


## Pull image data through API:

In [ ]:

print(api_helper.selection['run_title'])

# api_helper.get_image_content(api_helper.selection['run_root_id'], api_helper.selection['run'])
api_helper.get_image_arrays(api_helper.selection['run_root_id'], api_helper.selection['run'])
api_helper.get_crops()


# Findings

Run on :<br>

Positive id’s:<br>
False Positive id’s:<br>
Missed id's:<br>

Profiling applied to assist species confirmed identification, and derive potential re-run swatches.<br>

Swatch Range was: 


## Profiling results

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image , ImageDraw


def plot_colors(cropped):

    img =  Image.fromarray(np.uint8(cropped), 'RGB')
    plt.rcParams['figure.figsize'] = [16,5]

    plt.subplot(1,4,1)
    plt.axis('off')
    plt.title(f"Cropped {img.width} x {img.height}")
    plt.imshow(img)

    # work to corner offset on typical 86x86 crop
    left = 21 
    upper = 21
    right = 64
    lower = 64

    if left > img.width or right > img.width:
        left = 8
        right = img.width-8

    if upper > img.height or lower > img.height:
        upper = 8
        lower = img.height-8

    im_cropped = img.crop((left, upper,right,lower))
    mag = im_cropped.getcolors(img.width*img.height)
    mag = sorted(mag, key=lambda t: t[0], reverse=True)
    mag = [ ( np.linalg.norm(m[1]) , ([m[1][0]/255, m[1][1]/255, m[1][2]/255, 1] )) for m in mag]
    mag = sorted(mag, key=lambda t: t[0], reverse=True)[0:40]  
    colours = [ [m[1][0], m[1][1], m[1][2], m[1][3]] for m in mag]

    hst = im_cropped.histogram()
    # avoid dark/background bands & saturated 'whites'
    bmin = 32
    bmax = 246
    mkTop = 15
    rgb = ('r','g','b')
    slc = []
    colTgt = []
    for i, band in enumerate(rgb):
        offs = 256*i
        slc.append(hst[offs+bmin:offs+bmax])
    slcTop = np.max(slc)
    slcFloor = np.floor(slcTop/mkTop)
    for i, band in enumerate(rgb):
        rev= slc[i].copy()
        rev.reverse()
        tgt = next(x for x, val in enumerate(rev) if val > slcFloor)
        colTgt.append(bmax-tgt-bmin)

    plt.subplot(1,4,2)
    plt.axis('off')
    plt.title("Zoomed Area")
    imza = im_cropped.copy()
    ImageDraw.Draw(imza).rectangle([2,2,imza.width-3,imza.height-3], outline=(255,0,0))
    plt.imshow(imza)

    plt.subplot(1,4,3)
    plt.axis('off')
    plt.title("Brights Sampled")
    plt.pie([32 for i in colours], explode = [0.2 for i in colours],colors = colours, wedgeprops = {'linewidth': 0, 'edgecolor': None})     

    plt.subplot(1,4,4)
    plt.axis('off')
    imsw = im_cropped.copy()
    swc = (colTgt[0] + bmin, colTgt[1] +bmin, colTgt[2]+bmin)
    rngTgt = f"\n{swc}"
    ImageDraw.Draw(imsw).rectangle([2,2,imsw.width-3,(imsw.height/2)-3], fill=swc, outline=swc)
    swc = colours[-1]
    swc = (int(swc[0]*bmax), int(swc[1]*bmax), int(swc[2]*bmax))
    rngTgt = f"{rngTgt}\n{swc}"
    ImageDraw.Draw(imsw).rectangle([2,(imsw.height/2)+3,(imsw.width/2)-1,imsw.height-3], fill=swc, outline=swc)
    swc = colours[int(len(colours)/2)]
    swc = (int(swc[0]*bmax), int(swc[1]*bmax), int(swc[2]*bmax))
    rngTgt = f"{rngTgt}\n{swc}"
    ImageDraw.Draw(imsw).rectangle([(imsw.width/2)+1,(imsw.height/2)+3,imsw.width-3,imsw.height-3], fill=swc, outline=swc)
    plt.title(f"Usable Swatch? RGB:{rngTgt}")
    plt.imshow(imsw)

    plt.figure(figsize=(16,2))
    plt.xlim([0,bmax-bmin])
    plt.ylim([0,slcTop])
    plt.hlines(slcFloor,0,bmax-bmin)
    plt.vlines(colTgt,0,slcTop)
    plt.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom=False,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        labelbottom=False) # labels along the bottom edge are off
    plt.title("Brights Histogram")
    for i, band in enumerate(rgb):
        plt.plot(slc[i],band, linestyle='--', marker='o',markevery=True, scalex=False, scaley=False)
        plt.vlines(colTgt[i],0,slcTop,colors=[band])
    plt.show()


figures = api_helper.selection['run']

for ind, image_id in enumerate(figures):
    print("-----------------------------------------------------------------------")
    print(f"{figures[image_id]['display_name']}")
    print("-----------------------------------------------------------------------")
    plt.figure(figsize=(16,16))
    plt.axis('off')
    plt.imshow(figures[image_id]['array'])
    plt.title(figures[image_id]['display_name'])
    plt.show()

    for sub, cropped in enumerate(figures[image_id]['crops']):
        plot_colors(figures[image_id]['crops'][sub])
    #     break

    # break